<a href="https://colab.research.google.com/github/as9786/ComputerVision/blob/main/ObjectDetection/code/SPPNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SPPNet

## Bin

In [59]:
import math
import tensorflow as tf
import torch
import torch.nn as nn
from torch.nn import init
import functools
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F

In [53]:
a = tf.random.uniform(shape = [1,13,13,1])
a_shape = a.shape
a_shape[1]

13

In [54]:
def window_stride(shape,bin=3):
  s = shape[1]
  window_size = math.ceil(s / bin)
  stride = math.floor(s / bin)
  return window_size, stride

In [55]:
w, s = window_stride(a_shape)
w,s

(5, 4)

In [56]:
pool = tf.keras.layers.MaxPooling2D(pool_size=w,strides=s)

In [57]:
pool(a).shape

TensorShape([1, 3, 3, 1])

In [ ]:
b = tf.random.uniform(shape = [1,20,20,1])
b_shape = b.shape
b_shape[1]

20

In [ ]:
def window_stride(shape,bin=3):
  s = shape[1]
  window_size = math.ceil(s / bin)
  stride = math.floor(s / bin)
  return window_size, stride

In [ ]:
w, s = window_stride(b_shape)
w,s

(7, 6)

In [ ]:
pool2 = tf.keras.layers.MaxPooling2D(pool_size=w,strides=s)

In [ ]:
pool2(b).shape

TensorShape([1, 3, 3, 1])

## SPP layer

In [58]:
def spatial_pyramid_pool(self,previous_conv, num_sample, previous_conv_size, out_pool_size):  
    
    for i in range(len(out_pool_size)):
        
        h_wid = int(math.ceil(previous_conv_size[0] / out_pool_size[i]))
        w_wid = int(math.ceil(previous_conv_size[1] / out_pool_size[i]))
        h_pad = (h_wid*out_pool_size[i] - previous_conv_size[0] + 1)/2
        w_pad = (w_wid*out_pool_size[i] - previous_conv_size[1] + 1)/2
        maxpool = nn.MaxPool2d((h_wid, w_wid), stride=(h_wid, w_wid), padding=(h_pad, w_pad))
        x = maxpool(previous_conv)
        if(i == 0):
            spp = x.view(num_sample,-1)   
        else:
            spp = torch.cat((spp,x.view(num_sample,-1)), 1)
    return spp

In [ ]:
class SPP_NET(nn.Module):

    def __init__(self, opt, input_nc, ndf=64,  gpu_ids=[]):
        super(SPP_NET, self).__init__()
        self.gpu_ids = gpu_ids
        self.output_num = [4,2,1]
        
        self.conv1 = nn.Conv2d(input_nc, ndf, 4, 2, 1, bias=False)
        
        self.conv2 = nn.Conv2d(ndf, ndf * 2, 4, 1, 1, bias=False)
        self.BN1 = nn.BatchNorm2d(ndf * 2)

        self.conv3 = nn.Conv2d(ndf * 2, ndf * 4, 4, 1, 1, bias=False)
        self.BN2 = nn.BatchNorm2d(ndf * 4)

        self.conv4 = nn.Conv2d(ndf * 4, ndf * 8, 4, 1, 1, bias=False)
        self.BN3 = nn.BatchNorm2d(ndf * 8)

        self.conv5 = nn.Conv2d(ndf * 8, 64, 4, 1, 0, bias=False)
        self.fc1 = nn.Linear(10752,4096)
        self.fc2 = nn.Linear(4096,1000)

    def forward(self,x):
        x = self.conv1(x)
        x = self.LReLU1(x)

        x = self.conv2(x)
        x = F.leaky_relu(self.BN1(x))

        x = self.conv3(x)
        x = F.leaky_relu(self.BN2(x))
        
        x = self.conv4(x)
       
        spp = spatial_pyramid_pool(x,1,[int(x.size(2)),int(x.size(3))],self.output_num)
        # print(spp.size())
        fc1 = self.fc1(spp)
        fc2 = self.fc2(fc1)
        s = nn.Sigmoid()
        output = s(fc2)
        return output